In [2]:
from datasets import load_dataset

ds = load_dataset("luli0034/music-tags-to-spectrogram", split='train')

/home/akovel/Documents/HSE/Music-Predictor/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import numpy as np
import torch
from torch import nn
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, accuracy_score
from datasets import load_dataset
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

## Возьму только 10 процентов датасета, чтобы просто проверить, что модель может обучиться

In [ ]:
# subset_size = int(len(ds) * 0.5) 
# subset = ds.select(range(subset_size)) 

# print(subset)
# ds = subset

Dataset({
    features: ['image', 'text'],
    num_rows: 462
})


In [7]:
class MusicDataset(Dataset):
    def __init__(self, ds, transform=None):
        self.transform = transform
        self.data_frame = ds
    
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, index):
        genres = self.data_frame[index]["text"]
        if self.transform:
            image = self.transform(self.data_frame[index]["image"])
        return image, genres

In [8]:
device = torch.device("cuda")


In [9]:
def extract_image_features(dataloader, model):
    model.eval()
    features = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            output = model(inputs)
            features.append(output.cpu().numpy())
    return np.vstack(features)


In [10]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.linear(x)

### Поэтому я превращу их в квадрат

In [13]:
image_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = MusicDataset(ds, transform=image_transforms)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [12]:
model = models.resnet50(pretrained=True)
model.fc = nn.Identity()
model.to(device) 

/home/akovel/Documents/HSE/Music-Predictor/.conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/akovel/Documents/HSE/Music-Predictor/.conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
features = extract_image_features(dataloader, model)

OSError: unrecognized data stream contents when reading image file

### Преобразуем изображения, для одинакового вида

## Тренимся на изображениях. Смотрим в one hot encoding ЦЕЛИ жанров

In [ ]:
mlb = MultiLabelBinarizer()
all_genres = [genres for _, genres in dataset]
labels = mlb.fit_transform(all_genres)

In [ ]:
feature_tensor = torch.tensor(features, dtype=torch.float32).to(device)
labels_tensor = torch.tensor(labels, dtype=torch.float32).to(device)

### В Pytorch 

In [ ]:
model = LogisticRegressionModel(input_size=feature_tensor.shape[1]).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

/tmp/ipykernel_11220/1849083971.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(torch.stack(X_train), dtype=torch.float32)
/tmp/ipykernel_11220/1849083971.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(torch.stack(X_test), dtype=torch.float32)


### Baseline CV Train

In [ ]:
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(feature_tensor)

model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor.to(device))
    predictions = torch.sigmoid(outputs).cpu().numpy() > 0.5

print("Accuracy:", accuracy_score(y_test, predictions))

AttributeError: 'list' object has no attribute 'shape'

## Пока получили нулевую точность. Так, как вообще данных нет

## Напомню взял только 10 процентов

In [ ]:
print(classification_report(y_test, predictions, target_names=mlb.classes_))

              precision    recall  f1-score   support

                   1.00      1.00      1.00        28
           0       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           a       0.83      0.96      0.89        25
           b       0.40      0.15      0.22        13
           c       0.85      0.88      0.86        25
           d       0.56      0.36      0.43        14
           e       0.79      0.96      0.87        24
           f       0.00      0.00      0.00         3
           g       0.50      0.17      0.25         6
           h       0.50      0.27      0.35        11
           i       0.83      1.00      0.91        25
           j       0.00      0.00      0.00         0
           k       0.67      0.27      0.38        15
           l       0.73      0.96      0.83        23
           m       0.43      0.27      0.33        11
           n       0.86      0.96      0.91        25
           o       0.93    

/Users/akovel/anaconda3/envs/diploma/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/akovel/anaconda3/envs/diploma/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/akovel/anaconda3/envs/diploma/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi